In [1]:
from transformers import AutoTokenizer, LlamaForCausalLM
from transformers import LlamaTokenizer
import pandas as pd
import torch
import loralib as lora

In [ ]:

data = pd.read_csv("grammar_fixed.csv")
wrong_sentences = data['wrong_sentences'].to_list()
fixed_sentences = data['fixed_sentences'].to_list()
results = data['result'].to_list()

#sharpbai/vicuna-7b-v1.3
#decapoda-research/llama-7b-hf

model_name = 'sharpbai/vicuna-7b-v1.3'
model = LlamaForCausalLM.from_pretrained(model_name).to(torch.bfloat16).to("cuda")
tokenizer = LlamaTokenizer.from_pretrained(model_name)

prompt = "Hey, are you consciours? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
# generate_ids = model.generate(inputs.input_ids.to('cuda'), max_length=30)
generate_ids = model.generate(inputs.input_ids.to('cuda'), max_length=30)
return_text = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(return_text)

'''
train
'''

lora.mark_only_lora_as_trainable(model)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
epochs = 30
for epoch in range(epochs):
    for wrong_sentence, fixed_sentence, result in zip(wrong_sentences, fixed_sentences, results):
        instruction = "instruction : if there are grammatical errors in the sentence, correct them and make the sentence again. and explain the result.\n\n"
        prompt = instruction + 'input : \n\noriginal Sentence : "' + wrong_sentence + '" \n\nResult : "' + fixed_sentence +'"\n\nExplanation : ' + result
        inputs = tokenizer(prompt, return_tensors="pt")
        ouputs = model(inputs.input_ids.to('cuda'), labels = inputs.input_ids.to('cuda'))
        optimizer.zero_grad()
        ouputs.loss.backward()
        optimizer.step()

torch.save(lora.lora_state_dict(model), "llama.pt")


Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at sharpbai/vicuna-7b-v1.3 and are newly initialized: ['model.layers.12.self_attn.q_proj.lora_A', 'model.layers.1.self_attn.v_proj.lora_B', 'model.layers.4.self_attn.q_proj.lora_B', 'model.layers.25.self_attn.q_proj.lora_B', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.24.self_attn.q_proj.lora_B', 'model.layers.22.self_attn.q_proj.lora_A', 'model.layers.5.self_attn.v_proj.lora_B', 'model.layers.4.self_attn.v_proj.lora_B', 'model.layers.4.self_attn.q_proj.lora_A', 'model.layers.26.self_attn.v_proj.lora_A', 'model.layers.24.self_attn.q_proj.lora_A', 'model.layers.5.self_attn.q_proj.bias', 'model.layers.21.self_attn.v_proj.bias', 'model.layers.19.self_attn.v_proj.lora_A', 'model.layers.20.self_attn.q_proj.lora_B', 'model.layers.7.self_attn.q_proj.bias', 'model.layers.1.self_attn.q_proj.lora_A', 'model.layers.6.self_attn.q_proj.lora_A', 'model.layers.4.self_attn.v_proj.bias', 'model.layers.19.self_attn

Hey, are you consciours? Can you talk to me?
I'm a bit of a conspiracy theorist myself
